# Unstructured Data Ingestion and Processing With Ray Data

**Time to complete**: 35 min | **Difficulty**: Advanced | **Prerequisites**: Data engineering experience, document processing, basic NLP knowledge

## What you'll build

Build a comprehensive document ingestion pipeline that transforms unstructured documents from data lakes into structured, analytics-ready datasets using Ray Data's distributed processing capabilities for enterprise data warehouse workflows.


## Table of Contents

1. [Data Lake Document Discovery](#step-1-data-lake-document-discovery) (8 min)
2. [Document Processing and Classification](#step-2-document-processing-and-classification) (10 min)
3. [Text Extraction and Enrichment](#step-3-text-extraction-and-enrichment) (8 min)
4. [LLM-Powered Content Analysis](#step-4-llm-powered-content-analysis) (6 min)
5. [Data Warehouse Output](#step-5-data-warehouse-output) (3 min)


## Learning Objectives

**Why unstructured data ingestion matters**: Enterprise data lakes contain vast amounts of unstructured documents (PDFs, Word docs, presentations, reports) that need systematic processing to extract business value for analytics and reporting.

**Ray Data's ingestion capabilities**: Distribute document processing across clusters to handle large-scale document collections, extract structured data, and prepare analytics-ready datasets for data warehouse consumption.

**Data lake to warehouse patterns**: Techniques used by data engineering teams to systematically process document collections, extract structured information, and create queryable datasets for business intelligence.

**Production ingestion workflows**: Scalable document processing patterns that handle diverse file formats, extract metadata, and create structured schemas for downstream analytics systems.

**LLM integration strategies**: Document processing workflows that can use advanced analysis for content extraction from unstructured text.


## Overview

**Challenge**: Enterprise data lakes contain millions of unstructured documents (PDFs, Word docs, presentations) across multiple formats that need systematic processing to extract business value. Traditional document processing approaches struggle with:
- **Scale**: Single-machine processing limits document volume
- **Consistency**: Manual extraction creates inconsistent schemas  
- **Integration**: Complex infrastructure for analysis
- **Warehouse integration**: Manual data modeling and ETL processes

**Solution**: Ray Data enables end-to-end document ingestion pipelines:

| Pipeline Stage | Traditional Approach | Ray Data Approach | Benefit |
|------------------|-----------------------|---------------------|-----------|
| **Document Discovery** | Sequential file listing | Parallel `read_binary_files()` | Process millions of files |
| **Text Extraction** | Single-threaded parsing | Distributed `map_batches()` | Extract from all docs simultaneously |
| **Content Analysis** | Manual processing | Distributed analysis | Built-in batch processing |
| **Data Warehouse** | Custom ETL scripts | Native `write_parquet()` with partitioning | Production-ready output |

**Data Lake to Warehouse Flow**: This template demonstrates a complete pipeline from raw documents in data lakes to structured, queryable datasets ready for business intelligence and analytics workflows using Ray Data native operations.


## Prerequisites Checklist

Before starting, ensure you have:
- [ ] Understanding of data lake and data warehouse concepts
- [ ] Experience with document processing and text extraction
- [ ] Knowledge of structured data formats (Parquet, Delta Lake, Iceberg)
- [ ] Python environment with Ray Data and document processing libraries
- [ ] Access to S3 or other cloud storage for document sources


## Quick start (3 minutes)

This section demonstrates large-scale document ingestion using Ray Data:


In [1]:
import json
import logging
import uuid
from datetime import datetime
from pathlib import Path
from typing import Dict, Any, List

import numpy as np
import pandas as pd
import ray

# Configure Ray Data 
ctx = ray.data.DataContext.get_current()
ctx.enable_progress_bars = False
ctx.enable_operator_progress_bars = False

# Initialize Ray for distributed processing
ray.init(ignore_reinit_error=True)

2025-10-11 00:17:23,060	INFO worker.py:1833 -- Connecting to existing Ray cluster at address: 10.0.48.117:6379...
2025-10-11 00:17:23,071	INFO worker.py:2004 -- Connected to Ray cluster. View the dashboard at https://session-77uweunq3awbhqefvry4lwcqq5.i.anyscaleuserdata.com 
2025-10-11 00:17:23,075	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_d09a1f3a380b650bc6804514c9ba098775a62b40.zip' (1.11MiB) to Ray cluster...
2025-10-11 00:17:23,080	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_d09a1f3a380b650bc6804514c9ba098775a62b40.zip'.
/home/ray/anaconda3/lib/python3.12/site-packages/ray/_private/worker.py:2052: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(


Python version:,3.12.11
Ray version:,2.50.0
Dashboard:,http://session-77uweunq3awbhqefvry4lwcqq5.i.anyscaleuserdata.com


## Step 1: Data Lake Document Discovery

### Discover document collections in data lake


In [2]:

# Load document collection from data lake
document_collection = ray.data.read_binary_files(
    "s3://anyscale-rag-application/1000-docs/",
    include_paths=True,
    ray_remote_args={"num_cpus":0.025}  # High I/O concurrency for large document collections
).limit(100)

print(f"Dataset schema: {document_collection.schema()}")

2025-10-11 00:17:23,668	INFO logging.py:293 -- Registered dataset logger for dataset dataset_72_0
2025-10-11 00:17:23,686	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_72_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:17:23,687	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_72_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=1] -> TaskPoolMapOperator[ReadFiles]
2025-10-11 00:17:23,692	WARNING resource_manager.py:134 -- ⚠️  Ray's object store is configured to use only 27.9% of available memory (98.3GiB out of 352.0GiB total). For optimal Ray Data performance, we recommend setting the object store to at least 50% of available memory. You can do this by setting the 'object_store_memory' parameter when calling ray.init() or by setting the RAY_DEFAULT_OBJECT_STORE_MEMORY_PROPORTION environment variable.
/home/ray/anaconda3/lib/python3.12/site-packages/ray/anysca

Dataset schema: Column  Type
------  ----
bytes   binary
path    string


### Document metadata extraction


In [3]:
def process_file(record: Dict[str, Any]) -> Dict[str, Any]:
    """
    Extract text content from document files.
    
    Processes the bytes field immediately to avoid passing large binary data
    through multiple Ray Data operations. Returns basic file metadata and
    extracted text.
    """
    import io
    from pathlib import Path
    from unstructured.partition.auto import partition
    
    file_path = Path(record["path"])
    file_bytes = record["bytes"]
    file_size = len(file_bytes)
    file_extension = file_path.suffix.lower()
    file_name = file_path.name
    
    # Only process supported file extensions
    supported_extensions = {".pdf", ".docx", ".doc", ".pptx", ".ppt", ".html", ".txt"}
    
    if file_extension not in supported_extensions:
        return {
            "document_id": str(uuid.uuid4()),
            "file_path": str(file_path),
            "file_name": file_name,
            "file_extension": file_extension,
            "file_size_bytes": file_size,
            "file_size_mb": round(file_size / (1024 * 1024), 2),
            "discovery_timestamp": datetime.now().isoformat(),
            "extracted_text": "",
            "text_length": 0,
            "word_count": 0,
            "extraction_status": "unsupported_format"
        }
    
    try:
        with io.BytesIO(file_bytes) as stream:
            elements = partition(file=stream)
            
            # Combine all text elements
            extracted_text = " ".join([str(el) for el in elements]).strip()
            text_length = len(extracted_text)
            word_count = len(extracted_text.split()) if extracted_text else 0
            extraction_status = "success"
            
    except Exception as e:
        print(f"Cannot process file {file_path}: {e}")
        extracted_text = ""
        text_length = 0
        word_count = 0
        extraction_status = f"error: {str(e)[:100]}"
    
    return {
        "document_id": str(uuid.uuid4()),
        "file_path": str(file_path),
        "file_name": file_name,
        "file_extension": file_extension,
        "file_size_bytes": file_size,
        "file_size_mb": round(file_size / (1024 * 1024), 2),
        "discovery_timestamp": datetime.now().isoformat(),
        "extracted_text": extracted_text,
        "text_length": text_length,
        "word_count": word_count,
        "extraction_status": extraction_status
    }

# Apply text extraction
print("Extracting text from documents...")
documents_with_text = document_collection.map(
    process_file,
    concurrency=8,
    num_cpus=1
)


2025-10-11 00:17:30,138	INFO logging.py:293 -- Registered dataset logger for dataset dataset_74_0
2025-10-11 00:17:30,143	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_74_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:17:30,144	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_74_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Project] -> AggregateNumRows[AggregateNumRows]


Extracting text from documents...
(autoscaler +13s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.


(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P43' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Cannot set gray non-stroke color because /'P47' is an invalid float value
(Map(process_file) pid=17419, ip=10.0.34.16) Can

Text extraction completed: 100 documents processed


In [4]:
documents_with_text.limit(25).to_pandas()

2025-10-11 00:19:01,463	INFO logging.py:293 -- Registered dataset logger for dataset dataset_75_0
2025-10-11 00:19:01,467	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_75_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:19:01,468	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_75_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=25] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)]
/home/ray/anaconda3/lib/python3.12/site-packages/ray/data/_internal/execution/operators/task_pool_map_operator.py:165: UserWarning: The maximum number of concurrent tasks for 'Map(process_file)' is set to 8, but the operator only received 4 input(s). This means that the operator can launch at most 4 task(s), which is less than the concurrency limit. You might be able to increase the number of concurrent tasks by configuring `override_num_blocks` earlier in t

,document_id,file_path,file_name,file_extension,file_size_bytes,file_size_mb,discovery_timestamp,extracted_text,text_length,word_count,extraction_status
0,b1036612-8d10-49b2-a3f8-218af0493e8b,anyscale-rag-application/1000-docs/100G Networ...,100G Networking Technology Overview - Slides -...,.pdf,1516903,1.45,2025-10-11T00:19:05.131495,100G Networking Technology Overview Christophe...,8996,1558,success
1,062705dd-e55c-4257-b6b3-acdbff5ca43d,anyscale-rag-application/1000-docs/Grand Centr...,Grand Central Dispatch - FreeBSD Dev Summit (1...,.pdf,130189,0.12,2025-10-11T00:19:05.495427,Grand Central Dispatch FreeBSD Devsummit Rober...,7831,1071,success
2,d7b9d924-6f08-4417-99b6-5647a8ac7079,anyscale-rag-application/1000-docs/Monitor_a_j...,Monitor_a_job.docx,.docx,387461,0.37,2025-10-11T00:19:06.257774,Monitor a job Anyscale jobs provides several t...,3296,585,success
3,13311876-9e19-4204-ad2a-54332be9426d,anyscale-rag-application/1000-docs/Serial Orde...,Serial Order - A Parallel Distributed Processi...,.pdf,2281776,2.18,2025-10-11T00:19:11.156516,SERIAL ORDER: A PARALLEL DISTRmUTED PROCESSING...,132375,21122,success
4,57522ab2-e67e-41b5-9a63-76f738c8da16,anyscale-rag-application/1000-docs/jargn10-the...,jargn10-thejargonfilever00038gut.txt,.txt,1140873,1.09,2025-10-11T00:19:13.671480,This Is The Project Gutenberg Etext of The Hac...,1065517,170519,success
5,3513ce06-840d-4ffe-a6d6-e96a8c970ace,anyscale-rag-application/1000-docs/A Compariso...,A Comparison of Programming Languages in Econo...,.pdf,211355,0.20,2025-10-11T00:19:09.853818,A Comparison of Programming Languages in Econo...,33839,5307,success
6,106f0fda-642f-4dbd-a8b6-75ed920c9e61,anyscale-rag-application/1000-docs/A Compariso...,A Comparison of Software and Hardware Techniqu...,.pdf,156844,0.15,2025-10-11T00:19:11.700186,A Comparison of Software and Hardware Techniqu...,71494,11296,success
7,81f48d32-a87a-4e50-bafb-dba914a76c3f,anyscale-rag-application/1000-docs/A Compilati...,A Compilation Target for Probabilistic Program...,.pdf,892594,0.85,2025-10-11T00:19:13.122018,A Compilation Target for Probabilistic Program...,39374,6122,success
8,f6a01e8a-3708-496b-b2cb-2d487aa6fd8f,anyscale-rag-application/1000-docs/Graph Theor...,Graph Theory (2005).pdf,.pdf,206383,0.20,2025-10-11T00:19:13.693472,V. Adamchik Graph Theory Victor Adamchik Fall ...,10103,1600,success
9,7c655f32-9bde-488f-81bd-36fe0d93063c,anyscale-rag-application/1000-docs/Multidigit ...,Multidigit Multiplication for Mathematicians (...,.pdf,346439,0.33,2025-10-11T00:19:15.065682,MULTIDIGIT MULTIPLICATION FOR MATHEMATICIANS D...,60434,10046,success


In [5]:

def enrich_business_metadata(record: Dict[str, Any]) -> Dict[str, Any]:
    """
    Classify documents by business category and assign processing priority.
    
    This is a separate stage that operates on already-extracted text,
    performing pure metadata enrichment based on filename patterns.
    """
    file_name = record["file_name"]
    filename_lower = file_name.lower()
    file_size = record["file_size_bytes"]
    
    # Business classification for data warehouse categorization
    if any(keyword in filename_lower for keyword in ["financial", "earnings", "revenue", "profit"]):
        doc_type = "financial_document"
        business_category = "finance"
    elif any(keyword in filename_lower for keyword in ["legal", "contract", "agreement", "terms"]):
        doc_type = "legal_document"
        business_category = "legal"
    elif any(keyword in filename_lower for keyword in ["regulatory", "compliance", "filing", "sec"]):
        doc_type = "regulatory_document"
        business_category = "compliance"
    elif any(keyword in filename_lower for keyword in ["client", "customer", "portfolio"]):
        doc_type = "client_document"
        business_category = "client_services"
    elif any(keyword in filename_lower for keyword in ["market", "research", "analysis", "report"]):
        doc_type = "research_document"
        business_category = "research"
    else:
        doc_type = "general_document"
        business_category = "general"
    
    # Processing priority for workflow optimization
    if any(keyword in filename_lower for keyword in ["urgent", "critical", "deadline"]):
        priority = "high"
        priority_score = 3
    elif any(keyword in filename_lower for keyword in ["important", "quarterly", "annual"]):
        priority = "medium"
        priority_score = 2
    else:
        priority = "low"
        priority_score = 1
    
    return {
        **record,
        "document_type": doc_type,
        "business_category": business_category,
        "processing_priority": priority,
        "priority_score": priority_score,
        "estimated_pages": max(1, file_size // 50000),
        "processing_status": "classified"
    }


# Apply business metadata enrichment
print("\nEnriching with business metadata...")
documents_with_metadata = documents_with_text.map(
    enrich_business_metadata,
    concurrency=10,
    num_cpus=0.25
)


2025-10-11 00:19:29,691	INFO logging.py:293 -- Registered dataset logger for dataset dataset_77_0
2025-10-11 00:19:29,696	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_77_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:19:29,697	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_77_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> TaskPoolMapOperator[Project] -> AggregateNumRows[AggregateNumRows]



Enriching with business metadata...


/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/cluster_autoscaler/productivity_calculator.py:174: RuntimeWarning: invalid value encountered in divide
  gpu_fraction_per_op = (optimal_num_tasks_per_op * num_gpus_per_op) / np.sum(
(Map(process_file) pid=16615, ip=10.0.20.152) Cannot set gray non-stroke color because /'P12' is an invalid float value
(Map(process_file) pid=16615, ip=10.0.20.152) Cannot set gray non-stroke color because /'P45' is an invalid float value
(Map(process_file) pid=16615, ip=10.0.20.152) Cannot set gray non-stroke color because /'P69' is an invalid float value
(Map(process_file) pid=16615, ip=10.0.20.152) Cannot set gray non-stroke color because /'P90' is an invalid float value
(Map(process_file) pid=16615, ip=10.0.20.152) Cannot set gray non-stroke color because /'P111' is an invalid float value
(Map(process_file) pid=16615, ip=10.0.20.152) Cannot set gray non-stroke color because /'P132' is an invalid float value
(Map(process_file) 

Metadata enrichment completed: 100 documents classified


In [6]:
documents_with_metadata.limit(5).to_pandas()

2025-10-11 00:20:58,797	INFO logging.py:293 -- Registered dataset logger for dataset dataset_78_0
2025-10-11 00:20:58,801	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_78_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:20:58,802	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_78_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=5] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)]
/home/ray/anaconda3/lib/python3.12/site-packages/ray/data/_internal/execution/operators/task_pool_map_operator.py:165: UserWarning: The maximum number of concurrent tasks for 'Map(process_file)' is set to 8, but the operator only received 1 input(s). This means that the operator can launch at most 1 task(s), which is less than the concurrency limit. You might be able to increase the number of concurrent ta

,document_id,file_path,file_name,file_extension,file_size_bytes,file_size_mb,discovery_timestamp,extracted_text,text_length,word_count,extraction_status,document_type,business_category,processing_priority,priority_score,estimated_pages,processing_status
0,ffb14e1e-b5ab-4d7f-9a11-d41f47e66d30,anyscale-rag-application/1000-docs/100G Networ...,100G Networking Technology Overview - Slides -...,.pdf,1516903,1.45,2025-10-11T00:21:00.779167,100G Networking Technology Overview Christophe...,8996,1558,success,general_document,general,low,1,30,classified
1,7c1a9e64-0180-414c-8a40-f72404485064,anyscale-rag-application/1000-docs/Grand Centr...,Grand Central Dispatch - FreeBSD Dev Summit (1...,.pdf,130189,0.12,2025-10-11T00:21:01.153132,Grand Central Dispatch FreeBSD Devsummit Rober...,7831,1071,success,general_document,general,low,1,2,classified
2,43aaff7e-34d0-4042-b2c1-749bab93ce81,anyscale-rag-application/1000-docs/Monitor_a_j...,Monitor_a_job.docx,.docx,387461,0.37,2025-10-11T00:21:01.950850,Monitor a job Anyscale jobs provides several t...,3296,585,success,general_document,general,low,1,7,classified
3,e4a7205e-0af4-487c-8184-3e8e8e2732a7,anyscale-rag-application/1000-docs/Serial Orde...,Serial Order - A Parallel Distributed Processi...,.pdf,2281776,2.18,2025-10-11T00:21:06.921489,SERIAL ORDER: A PARALLEL DISTRmUTED PROCESSING...,132375,21122,success,general_document,general,low,1,45,classified
4,bd194fc1-0137-4fe8-b336-c32a59f0fd94,anyscale-rag-application/1000-docs/jargn10-the...,jargn10-thejargonfilever00038gut.txt,.txt,1140873,1.09,2025-10-11T00:21:09.486768,This Is The Project Gutenberg Etext of The Hac...,1065517,170519,success,general_document,general,low,1,22,classified


In [7]:
# Use Ray Data native operations for document collection analysis
from ray.data.aggregate import Count, Sum, Mean, Max, Min

print("Analyzing document collection using Ray Data native operations...")

# Document type distribution using native groupby
doc_type_stats = documents_with_metadata.groupby("document_type").aggregate(
    Count(),
    Sum("file_size_bytes"),
    Mean("file_size_mb"),
    Max("estimated_pages")
)


2025-10-11 00:21:09,660	INFO logging.py:293 -- Registered dataset logger for dataset dataset_81_0
/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/util/dependencies.py:42: UserWarning: Numba isn't available. Install numba>=0.61>=0.61 to get better performance for hash partitioning operations. Falling back to slower Python implementation for RayTurbo optimizations.
  warnings.warn(
2025-10-11 00:21:09,668	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_81_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:21:09,669	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_81_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> HashAggregateOperator[HashAggregate(key_columns=('document_type',), num_partitions=200)] -

Analyzing document collection using Ray Data native operations...
Document Type Distribution:


/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/cluster_autoscaler/productivity_calculator.py:174: RuntimeWarning: invalid value encountered in divide
  gpu_fraction_per_op = (optimal_num_tasks_per_op * num_gpus_per_op) / np.sum(
(Map(process_file) pid=17503, ip=10.0.4.21) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=17503, ip=10.0.4.21) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=17503, ip=10.0.4.21) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=17503, ip=10.0.4.21) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=17503, ip=10.0.4.21) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=17503, ip=10.0.4.21) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pid=17503, ip=

       document_type  count()  sum(file_size_bytes)  mean(file_size_mb)  \
0   general_document       99              91471983            0.881515   
1  research_document        1                432535            0.410000   

   max(estimated_pages)  
0                   159  
1                     8  


In [8]:

# Business category analysis
category_stats = documents_with_metadata.groupby("business_category").aggregate(
    Count(),
    Mean("priority_score"),
    Sum("file_size_mb")
)

2025-10-11 00:22:43,832	INFO logging.py:293 -- Registered dataset logger for dataset dataset_84_0
2025-10-11 00:22:43,926	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_84_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:22:43,926	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_84_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> HashAggregateOperator[HashAggregate(key_columns=('business_category',), num_partitions=200)] -> LimitOperator[limit=5]


Business Category Analysis:


/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/cluster_autoscaler/productivity_calculator.py:174: RuntimeWarning: invalid value encountered in divide
  gpu_fraction_per_op = (optimal_num_tasks_per_op * num_gpus_per_op) / np.sum(
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pid=16559, ip=

  business_category  count()  mean(priority_score)  sum(file_size_mb)
0          research        1                   1.0               0.41
1           general       99                   1.0              87.27


## Step 2: Document Processing and Classification

###  Text extraction and quality assessment


In [9]:
from ray.data.expressions import col, lit

def assess_document_quality(record: Dict[str, Any]) -> Dict[str, Any]:
    """Assess document quality for data warehouse ingestion."""
    
    quality_score = 0
    quality_issues = []
    
    if record["file_size_mb"] > 0.01:
        quality_score += 1
    else:
        quality_issues.append("file_too_small")
    
    if record["text_length"] > 100:
        quality_score += 1
    else:
        quality_issues.append("insufficient_text")
    
    if record["business_category"] != "general":
        quality_score += 1
    else:
        quality_issues.append("low_business_relevance")
    
    if record["word_count"] > 20:
        quality_score += 1
    else:
        quality_issues.append("insufficient_content")
    
    quality_rating = "high" if quality_score >= 4 else "medium" if quality_score >= 2 else "low"
    
    return {
        **record,
        "quality_score": quality_score,
        "quality_rating": quality_rating,
        "quality_issues": json.dumps(quality_issues)
    }

# Apply quality assessment (text extraction already done in previous step)
quality_assessed_docs = documents_with_metadata.map_batches(
    process_quality_assessment_batch,
    num_cpus=0.25,
    batch_size=2000
)


/tmp/ipykernel_67923/1159361854.py:71: DeprecationWarning: String expressions are deprecated and will be removed in a future version. Use predicate expressions from ray.data.expressions instead. For example: from ray.data.expressions import col; ds.filter(expr=col('column_name') > 5)
  high_quality_docs = quality_assessed_docs.filter(
2025-10-11 00:24:18,026	INFO logging.py:293 -- Registered dataset logger for dataset dataset_87_0
2025-10-11 00:24:18,032	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_87_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:24:18,033	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_87_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> TaskPoolMapOperator[MapBatches(process_quality_assessment_batch)] -

Assessing document quality...


/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/cluster_autoscaler/productivity_calculator.py:174: RuntimeWarning: invalid value encountered in divide
  gpu_fraction_per_op = (optimal_num_tasks_per_op * num_gpus_per_op) / np.sum(
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P26' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P42' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P54' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P62' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P72' is an invalid float value
(Map(process_file) pid=16559, ip=10.0.6.91) Cannot set gray non-stroke color because /'P80' is an invalid float value
(Map(process_file) pid=16559, ip=

Total documents assessed: 100


(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P63' is an invalid float value [repeated 90x across cluster]
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P2640' is an invalid float value [repeated 575x across cluster]
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P352' is an invalid float value [repeated 473x across cluster]
(Map(process_file) pid=16216, ip=10.0.18.28) Cannot set gray non-stroke color because /'P157' is an invalid float value [repeated 142x across cluster]
(Map(process_file) pid=15818, ip=10.0.0.255) Cannot set gray non-stroke color because /'H3' is an invalid float value [repeated 106x across cluster]
2025-10-11 00:26:13,979	WARNING issue_detector_manager.py:58 -- 

Operator 'ReadFiles' uses 717.8MB of memory per task on average, but
Ray only requests 0.0B per task at the start of the pipeline.

To avoid out-of-memory errors, consider setting 

High quality documents: 1


## Step 3: Text Chunking and Enrichment


In [10]:
def create_text_chunks(record: Dict[str, Any]) -> List[Dict[str, Any]]:
    """Create text chunks optimized for processing and analytics."""
    
    text = record["extracted_text"]
    chunk_size = 1500
    overlap = 150
    
    chunks = []
    start = 0
    chunk_index = 0
    
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunk_text = text[start:end]
        
        chunk_record = {
            **record,
            "chunk_id": str(uuid.uuid4()),
            "chunk_index": chunk_index,
            "chunk_text": chunk_text,
            "chunk_length": len(chunk_text),
            "chunk_word_count": len(chunk_text.split())
        }
        
        chunks.append(chunk_record)
        
        # If we've reached the end of the text, stop
        if end >= len(text):
            break
            
        start = end - overlap
        chunk_index += 1
    
    # Update total chunks
    for chunk in chunks:
        chunk["total_chunks"] = len(chunks)
    
    return chunks

# Apply text chunking using Ray Data flat_map
print("Creating text chunks...")

chunked_documents = quality_assessed_docs.flat_map(
    create_text_chunks,
    num_cpus=0.5
)

2025-10-11 00:27:23,336	INFO logging.py:293 -- Registered dataset logger for dataset dataset_90_0
2025-10-11 00:27:23,342	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_90_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:27:23,343	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_90_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> TaskPoolMapOperator[MapBatches(process_quality_assessment_batch)] -> TaskPoolMapOperator[FlatMap(create_text_chunks)] -> TaskPoolMapOperator[Project] -> AggregateNumRows[AggregateNumRows]


Creating text chunks...


(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P43' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Cannot set gray non-stroke color because /'P47' is an invalid float value
(Map(process_file) pid=15723, ip=10.0.0.100) Can

Text chunking completed: 5,116 chunks created


## Step 4: Data Warehouse Schema and Output

### Create data warehouse schema


In [15]:
from datetime import datetime

# Apply warehouse schema transformation using expressions API
print("Creating data warehouse schema...")

processing_date = datetime.now().isoformat()[:10]

warehouse_dataset = chunked_documents.select_columns([
    # Primary identifiers
    "document_id",
    "chunk_id",
    
    # Dimensional attributes
    "business_category",
    "document_type",
    "file_extension",
    "quality_rating",
    "processing_priority",
    
    # Fact measures
    "file_size_mb",
    "word_count",
    "chunk_word_count",
    "quality_score",
    "priority_score",
    "estimated_pages",
    "chunk_index",
    "total_chunks",
    
    # Content fields
    "chunk_text",
    "file_name",
    "file_path",
    
    # Existing metadata
    "discovery_timestamp",
    "extraction_status",
    "processing_status"
]).rename_columns({
    "chunk_text": "text_content"
}).add_column(
    "processing_date", lambda df: processing_date
).add_column(
    "pipeline_version", lambda df: "1.0"
).add_column(
    "processing_engine", lambda df: "ray_data"
)

2025-10-11 00:50:59,250	INFO logging.py:293 -- Registered dataset logger for dataset dataset_101_0
2025-10-11 00:50:59,257	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_101_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:50:59,257	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_101_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> TaskPoolMapOperator[MapBatches(process_quality_assessment_batch)] -> TaskPoolMapOperator[FlatMap(create_text_chunks)] -> TaskPoolMapOperator[Project->MapBatches(add_column)->MapBatches(add_column)->MapBatches(add_column)->Project] -> AggregateNumRows[AggregateNumRows]
2025-10-11 00:50:59,285	WARNING progress_bar.py:120 -- Truncating long operator name to 100 characters. To disable this behavior, se

Creating data warehouse schema...


(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P43' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Cannot set gray non-stroke color because /'P47' is an invalid float value
(Map(process_file) pid=17489, ip=10.0.34.16) Can

Warehouse schema created: 5,116 records


### Write to data warehouse with partitioning


In [17]:
# Write main warehouse table with partitioning
print("Writing to data warehouse...")

OUTPUT_WAREHOUSE_PATH = "/mnt/cluster_storage"

warehouse_dataset.write_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/main_table/",
    partition_cols=["business_category", "processing_date"],
    compression="snappy",
    ray_remote_args={"num_cpus":0.1}
)

print("Main warehouse table written successfully")


2025-10-11 00:54:51,685	INFO logging.py:293 -- Registered dataset logger for dataset dataset_105_0
2025-10-11 00:54:51,692	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_105_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 00:54:51,693	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_105_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> TaskPoolMapOperator[MapBatches(process_quality_assessment_batch)] -> TaskPoolMapOperator[FlatMap(create_text_chunks)] -> TaskPoolMapOperator[Project->MapBatches(add_column)->MapBatches(add_column)->MapBatches(add_column)] -> TaskPoolMapOperator[Write]


Writing to data warehouse...


/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/cluster_autoscaler/productivity_calculator.py:174: RuntimeWarning: invalid value encountered in divide
  gpu_fraction_per_op = (optimal_num_tasks_per_op * num_gpus_per_op) / np.sum(
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=16352, ip=10.0.30.227) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pi

Main warehouse table written successfully


In [20]:
print("Creating business-specific datasets...")

# Financial documents dataset
financial_analytics = warehouse_dataset.filter(
    expr="business_category == 'finance'",
    num_cpus=0.1
).select_columns([
    "document_id", "chunk_id", "text_content", "summary", 
    "quality_score", "processing_date", "metrics_count"
])

financial_analytics.write_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/analytics/financial/",
    partition_cols=["processing_date"],
    compression="snappy",
    ray_remote_args={"num_cpus":0.1}
)

# Compliance documents dataset
compliance_analytics = warehouse_dataset.filter(
   expr="business_category == 'compliance'",
    num_cpus=0.1
).select_columns([
    "document_id", "chunk_id", "text_content", "summary",
    "quality_score", "content_priority", "processing_date"
])

compliance_analytics.write_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/analytics/compliance/",
    partition_cols=["processing_date"],
    compression="snappy",
    ray_remote_args={"num_cpus":0.1}
)


/tmp/ipykernel_67923/269082049.py:4: DeprecationWarning: String expressions are deprecated and will be removed in a future version. Use predicate expressions from ray.data.expressions instead. For example: from ray.data.expressions import col; ds.filter(expr=col('column_name') > 5)
  financial_analytics = warehouse_dataset.filter(
2025-10-11 01:01:53,536	INFO logging.py:293 -- Registered dataset logger for dataset dataset_114_0
2025-10-11 01:01:53,544	INFO streaming_executor.py:159 -- Starting execution of Dataset dataset_114_0. Full logs are in /tmp/ray/session_2025-10-10_21-11-45_497822_2529/logs/ray-data
2025-10-11 01:01:53,545	INFO streaming_executor.py:160 -- Execution plan of Dataset dataset_114_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ListFiles] -> LimitOperator[limit=100] -> TaskPoolMapOperator[ReadFiles] -> TaskPoolMapOperator[Map(process_file)] -> TaskPoolMapOperator[Map(enrich_business_metadata)] -> TaskPoolMapOperator[MapBatches(process_quality_assessment_batch)] ->

Creating business-specific datasets...


/home/ray/anaconda3/lib/python3.12/site-packages/ray/anyscale/data/_internal/cluster_autoscaler/productivity_calculator.py:174: RuntimeWarning: invalid value encountered in divide
  gpu_fraction_per_op = (optimal_num_tasks_per_op * num_gpus_per_op) / np.sum(
(Map(process_file) pid=16674, ip=10.0.20.152) Cannot set gray non-stroke color because /'P15' is an invalid float value
(Map(process_file) pid=16674, ip=10.0.20.152) Cannot set gray non-stroke color because /'P19' is an invalid float value
(Map(process_file) pid=16674, ip=10.0.20.152) Cannot set gray non-stroke color because /'P23' is an invalid float value
(Map(process_file) pid=16674, ip=10.0.20.152) Cannot set gray non-stroke color because /'P27' is an invalid float value
(Map(process_file) pid=16674, ip=10.0.20.152) Cannot set gray non-stroke color because /'P33' is an invalid float value
(Map(process_file) pid=16674, ip=10.0.20.152) Cannot set gray non-stroke color because /'P39' is an invalid float value
(Map(process_file) pi

(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'document_id': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'chunk_id': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'business_category': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'document_type': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'file_extension': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'quality_rating': cannot call `vectorize` on size 0 inputs unless `otypes` is set
(Project pid=16718, ip=10.0.0.100) Error calculating size for column 'processing_priority': cannot ca

### Create analytics summary tables


In [ ]:
print("Creating analytics summary tables...")

# Processing metrics by category and date
processing_metrics = warehouse_dataset.groupby(["business_category", "processing_date"]).aggregate(
    Count(),
    Sum("file_size_mb"),
    Mean("word_count"),
    Mean("quality_score")
)

processing_metrics.write_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/summaries/processing_metrics/",
    partition_cols=["processing_date"],
    compression="snappy",
    ray_remote_args={"num_cpus":0.1}
)

# Quality distribution analysis
quality_distribution = warehouse_dataset.groupby(["quality_rating", "business_category"]).aggregate(
    Count(),
    Mean("word_count"),
    Mean("entities_count"),
    Mean("metrics_count")
)

quality_distribution.write_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/summaries/quality_distribution/",
    compression="snappy",
    ray_remote_args={"num_cpus":0.1}
)



## Verification and Summary

### Verify data warehouse outputs


In [ ]:
# Verify warehouse outputs
print("Verifying data warehouse integration...")

# Read back main table
main_table_verify = ray.data.read_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/main_table/",
    num_cpus=0.025
)

# Read back summary tables
metrics_verify = ray.data.read_parquet(
    f"{OUTPUT_WAREHOUSE_PATH}/summaries/processing_metrics/",
    num_cpus=0.025
)

print(f"Data warehouse verification:")
print(f"  Main table records: {main_table_verify.count():,}")
print(f"  Processing metrics: {metrics_verify.count():,}")
print(f"  Schema compatibility: Verified")

# Display sample data
print("\\nSample warehouse records:")
samples = main_table_verify.take(10)
for i, record in enumerate(samples):
    print(f"  {i+1}. Doc: {record['document_id'][:8]}, Category: {record['business_category']}, "
          f"Words: {record['word_count']}, Quality: {record['quality_rating']}")


## Summary and Next Steps

This notebook demonstrates a complete document ingestion pipeline using Ray Data:

### Key Features Demonstrated

**Ray Data Operations**:
- `read_binary_files()` for large-scale document discovery
- `map()` and `map_batches()` for distributed processing
- `filter()` with expressions API for efficient filtering
- `flat_map()` for text chunking
- `groupby().aggregate()` for analytics
- `write_parquet()` with partitioning for data warehouse output

**CPU-Based Processing**:
- Pattern matching for content analysis
- No GPU requirements
- Scalable across CPU-only clusters

**Data Warehouse Integration**:
- Partitioned tables for query optimization
- Business-specific datasets
- Summary tables for analytics
- Schema standardization

### Enabling GPU-Accelerated LLM Processing

For GPU-accelerated content analysis with vLLM:

1. Install Ray Data LLM package: `pip install -U vllm==0.7.2`
2. Configure GPU resources in your cluster
3. Replace the CPU-based analysis in Step 4 with:

```python
from ray.data.llm import vLLMEngineProcessorConfig, build_llm_processor

llm_config = vLLMEngineProcessorConfig(
    model_source="unsloth/Llama-3.1-8B-Instruct",
    engine_kwargs={
        "max_model_len": 16384,
        "enable_chunked_prefill": True,
        "max_num_batched_tokens": 4096,
        "tensor_parallel_size": 1,
    },
    concurrency=1,
    batch_size=32,
    accelerator_type="A10G"
)

llm_processor = build_llm_processor(
    llm_config,
    preprocess=create_prompts,
    postprocess=extract_structured_data
)

analyzed_docs = llm_processor(chunked_documents)
```

### Production Recommendations

1. **Use real text extraction libraries**: PyPDF2, python-docx, python-pptx, BeautifulSoup
2. **Tune batch sizes**: Adjust based on document size and cluster resources
3. **Monitor progress**: Use Ray dashboard for performance visibility
4. **Scale horizontally**: Add workers to increase throughput
5. **Optimize partitioning**: Match partitioning strategy to query patterns

This pipeline transforms unstructured documents from data lakes into structured, analytics-ready datasets for enterprise data warehouse consumption and business intelligence workflows.
